# Create predictions and graph resultant time series

Using M5 data held in BQ, do a number of simple t.s estimations on sales and then plot outputs.

In [20]:
%load_ext google.cloud.bigquery
import numpy as np # linear algebra
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error
import plotly.express as px
import plotly.graph_objects as go

from fbprophet import Prophet

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [4]:
%%bigquery df_sales_store_CA_1
-- select sales from one store
select 
*
from
sales_train_evaluation.sales_train_evaluation a
where a.store_id = 'CA_1';

In [5]:
df_sales_store_CA_1.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
4,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0


In [6]:
rms_guess = []
rms_prophet = []
rms_croston = []
rms_croston_tsb = []

PREDICT_DAYS = 28

# just look at one item row
row = df_sales_store_CA_1.iloc[0:1,:]
row

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1


Unpivot the date columns

In [7]:
df_row = pd.DataFrame(row)
df_item = pd.melt(
    df_row,
    id_vars=[
        'id',
        'item_id',
        'dept_id',
        'cat_id',
        'store_id',
        'state_id'
    ],
    var_name='d',
    value_name='sold'
).dropna()

df_item

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0
2,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0
3,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0
4,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0
...,...,...,...,...,...,...,...,...
1936,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1937,0
1937,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1938,3
1938,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1939,3
1939,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1940,0


In [8]:
%%bigquery df_calendar
SELECT date,wm_yr_wk,weekday,wday,month,year,d
FROM `kaggle-covid.calendar.calendar`;

Convert the date codes in the item row to dates by merging with the calendar

In [9]:
df_item = pd.merge(df_item, df_calendar, on='d', how='left')
df_item.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,weekday,wday,month,year
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0,2011-01-30,11101,Sunday,2,1,2011
2,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,Monday,3,1,2011
3,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,Tuesday,4,2,2011
4,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,Wednesday,5,2,2011


In [10]:
df_item.shape

(1941, 14)

In [11]:
df_item['date'] = pd.to_datetime(df_item['date'], errors='coerce')

For time series estimation we only need date and target variable but we will keep identifier (id) as well for summarising.

In [13]:
df_train = df_item[['date', 'sold', 'id']]
df_train.columns = ["ds", "y", 'id']

# select the training range of dates 
#df_train.iloc[0:1913, :2 ]

# set up data frame to predict into
df_predict = df_train.iloc[1913: , 0:1 ]
df_predict

m = Prophet(daily_seasonality=True)
m.fit(df_train.iloc[0:1913, :2 ])

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [16]:
df_forecast = m.predict(df_predict)

df_forecast['yhat_ceil'] = np.floor(df_forecast['yhat'])
df_forecast['yhat_floor'] = np.floor(df_forecast['yhat'])
df_forecast[['ds','yhat', 'yhat_lower', 'yhat_upper', 'yhat_floor', 'yhat_ceil']]

,ds,yhat,yhat_lower,yhat_upper,yhat_floor,yhat_ceil
0,2016-04-25,0.762951,-0.040053,1.534401,0.0,0.0
1,2016-04-26,0.823022,0.030706,1.609355,0.0,0.0
2,2016-04-27,0.707239,-0.072136,1.471128,0.0,0.0
3,2016-04-28,0.697490,-0.113545,1.461463,0.0,0.0
4,2016-04-29,0.749813,-0.033114,1.538886,0.0,0.0
5,2016-04-30,0.896919,0.160119,1.678664,0.0,0.0
6,2016-05-01,0.773520,0.012468,1.580995,0.0,0.0
7,2016-05-02,0.721987,-0.023833,1.466441,0.0,0.0
8,2016-05-03,0.780765,0.011239,1.585561,0.0,0.0
9,2016-05-04,0.663588,-0.143811,1.391055,0.0,0.0


Actual 28 day sales numbers from evaluation set

In [21]:
# test data: select actual sales figures for the predicted date range
df_actuals = df_train.iloc[1913: , : ]

rms_prophet.append(sqrt(mean_squared_error(df_actuals['y'].astype('int32'), df_forecast['yhat'])))

For comparison: take a naive guess that previous month's sales would be the predicted month's sales

In [22]:
df_guess_last_month = df_train.iloc[1885:1913 , :2 ]

In [23]:
rms_guess.append(sqrt(mean_squared_error(df_actuals['y'].astype('int32'), df_guess_last_month['y'])))

In [24]:
print("Prophet error...")
print(sum(rms_prophet))
print("Guess error..")
print(sum(rms_guess))
print("Croston  error..")
print(sum(rms_croston))
print("Croston TSB error..")
print(sum(rms_croston_tsb))

Prophet error...
1.4573484899751152
Guess error..
1.6690459207925603
Croston  error..
0
Croston TSB error..
0
